# Step 2: Data Wrangling

This notebook reads in raw American Community Survey (ACS) and Child Opportunity Index (COI) data and performs several cleaning tasks to normalize the data, such as:
1. Renaming columns with descriptive headers
2. Removing columns with all missing data
3. Removing columns that contain data we do not need
4. Creating a consistently-formatted FIPS code variable
5. Merging the ACS and COI data

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import io, s3fs, json, tabula,os
os.chdir('/Users/sarahmartin/Documents/GT/Bleeding-Hearts/')
from modules import s3upload
pd.set_option('display.max_columns', None)
print('Program run at', dt.now())

Program run at 2021-06-13 18:57:22.593702


## Rename ACS columns
The raw ACS data has codes for column headers (see dataframe head below). We accessed the descriptive labels in Step 1. In the following steps, we clean the descriptive labels and replace the codes with the descriptive labels.

In [2]:
#Import ACS data and header data from AWS.
acs = pd.read_csv('s3://bleeding-hearts/rawdata/acs5.csv',
                 low_memory=False,
                 error_bad_lines=False)
acs.head()

b'Skipping line 35937: expected 1103 fields, saw 1795\n'


,Unnamed: 0,GEO_ID,DP03_0001E,DP03_0001M,DP03_0001PE,DP03_0001PM,DP03_0002E,DP03_0002M,DP03_0002PE,DP03_0002PM,DP03_0003E,DP03_0003M,DP03_0003PE,DP03_0003PM,DP03_0004E,DP03_0004M,DP03_0004PE,DP03_0004PM,DP03_0005E,DP03_0005M,DP03_0005PE,DP03_0005PM,DP03_0006E,DP03_0006M,DP03_0006PE,DP03_0006PM,DP03_0007E,DP03_0007M,DP03_0007PE,DP03_0007PM,DP03_0008E,DP03_0008M,DP03_0008PE,DP03_0008PM,DP03_0009E,DP03_0009M,DP03_0009PE,DP03_0009PM,DP03_0010E,DP03_0010M,DP03_0010PE,DP03_0010PM,DP03_0011E,DP03_0011M,DP03_0011PE,DP03_0011PM,DP03_0012E,DP03_0012M,DP03_0012PE,DP03_0012PM,DP03_0013E,DP03_0013M,DP03_0013PE,DP03_0013PM,DP03_0014E,DP03_0014M,DP03_0014PE,DP03_0014PM,DP03_0015E,DP03_0015M,DP03_0015PE,DP03_0015PM,DP03_0016E,DP03_0016M,DP03_0016PE,DP03_0016PM,DP03_0017E,DP03_0017M,DP03_0017PE,DP03_0017PM,DP03_0018E,DP03_0018M,DP03_0018PE,DP03_0018PM,DP03_0019E,DP03_0019M,DP03_0019PE,DP03_0019PM,DP03_0020E,DP03_0020M,DP03_0020PE,DP03_0020PM,DP03_0021E,DP03_0021M,DP03_0021PE,DP03_0021PM,DP03_0022E,DP03_0022M,DP03_0022PE,DP03_0022PM,DP03_0023E,DP03_0023M,DP03_0023PE,DP03_0023PM,DP03_0024E,DP03_0024M,DP03_0024PE,DP03_0024PM,DP03_0025E,DP03_0025M,DP03_0025PE,DP03_0025PM,DP03_0026E,DP03_0026M,DP03_0026PE,DP03_0026PM,DP03_0027E,DP03_0027M,DP03_0027PE,DP03_0027PM,DP03_0028E,DP03_0028M,DP03_0028PE,DP03_0028PM,DP03_0029E,DP03_0029M,DP03_0029PE,DP03_0029PM,DP03_0030E,DP03_0030M,DP03_0030PE,DP03_0030PM,DP03_0031E,DP03_0031M,DP03_0031PE,DP03_0031PM,DP03_0032E,DP03_0032M,DP03_0032PE,DP03_0032PM,DP03_0033E,DP03_0033M,DP03_0033PE,DP03_0033PM,DP03_0034E,DP03_0034M,DP03_0034PE,DP03_0034PM,DP03_0035E,DP03_0035M,DP03_0035PE,DP03_0035PM,DP03_0036E,DP03_0036M,DP03_0036PE,DP03_0036PM,DP03_0037E,DP03_0037M,DP03_0037PE,DP03_0037PM,DP03_0038E,DP03_0038M,DP03_0038PE,DP03_0038PM,DP03_0039E,DP03_0039M,DP03_0039PE,DP03_0039PM,DP03_0040E,DP03_0040M,DP03_0040PE,DP03_0040PM,DP03_0041E,DP03_0041M,DP03_0041PE,DP03_0041PM,DP03_0042E,DP03_0042M,DP03_0042PE,DP03_0042PM,DP03_0043E,DP03_0043M,DP03_0043PE,DP03_0043PM,DP03_0044E,DP03_0044M,DP03_0044PE,DP03_0044PM,DP03_0045E,DP03_0045M,DP03_0045PE,DP03_0045PM,DP03_0046E,DP03_0046M,DP03_0046PE,DP03_0046PM,DP03_0047E,DP03_0047M,DP03_0047PE,DP03_0047PM,DP03_0048E,DP03_0048M,DP03_0048PE,DP03_0048PM,DP03_0049E,DP03_0049M,DP03_0049PE,DP03_0049PM,DP03_0050E,DP03_0050PE,DP03_0050M,DP03_0050PM,DP03_0051E,DP03_0051M,DP03_0051PE,DP03_0051PM,DP03_0052E,DP03_0052M,DP03_0052PE,DP03_0052PM,DP03_0053E,DP03_0053M,DP03_0053PE,DP03_0053PM,DP03_0054E,DP03_0054M,DP03_0054PE,DP03_0054PM,DP03_0055E,DP03_0055M,DP03_0055PE,DP03_0055PM,DP03_0056E,DP03_0056M,DP03_0056PE,DP03_0056PM,DP03_0057E,DP03_0057M,DP03_0057PE,DP03_0057PM,DP03_0058E,DP03_0058M,DP03_0058PE,DP03_0058PM,DP03_0059E,DP03_0059M,DP03_0059PE,DP03_0059PM,DP03_0060E,DP03_0060M,DP03_0060PE,DP03_0060PM,DP03_0061E,DP03_0061M,DP03_0061PE,DP03_0061PM,DP03_0062E,DP03_0062M,DP03_0062PE,DP03_0062PM,DP03_0063E,DP03_0063M,DP03_0063PE,DP03_0063PM,DP03_0064E,DP03_0064M,DP03_0064PE,DP03_0064PM,DP03_0065E,DP03_0065M,DP03_0065PE,DP03_0065PM,DP03_0066E,DP03_0066M,DP03_0066PE,DP03_0066PM,DP03_0067E,DP03_0067M,DP03_0067PE,DP03_0067PM,DP03_0068E,DP03_0068M,DP03_0068PE,DP03_0068PM,DP03_0069E,DP03_0069M,DP03_0069PE,DP03_0069PM,DP03_0070E,DP03_0070M,DP03_0070PE,DP03_0070PM,DP03_0071E,DP03_0071M,DP03_0071PE,DP03_0071PM,DP03_0072E,DP03_0072M,DP03_0072PE,DP03_0072PM,DP03_0073E,DP03_0073M,DP03_0073PE,DP03_0073PM,DP03_0074E,DP03_0074M,DP03_0074PE,DP03_0074PM,DP03_0075E,DP03_0075M,DP03_0075PE,DP03_0075PM,DP03_0076E,DP03_0076M,DP03_0076PE,DP03_0076PM,DP03_0077E,DP03_0077M,DP03_0077PE,DP03_0077PM,DP03_0078E,DP03_0078M,DP03_0078PE,DP03_0078PM,DP03_0079E,DP03_0079M,DP03_0079PE,DP03_0079PM,DP03_0080E,DP03_0080M,DP03_0080PE,DP03_0080PM,DP03_0081E,DP03_0081M,DP03_0081PE,DP03_0081PM,DP03_0082E,DP03_0082M,DP03_0082PE,DP03_0082PM,DP03_0083E,DP03_0083M,DP03_0083PE,DP03_0083PM,DP03_0084E,DP03_0084M,DP03_0084PE,DP03_0084PM,DP03_0085E,DP03_0085M,DP03_0085PE,DP03_0085PM,DP03_0086E,DP03_0086M,DP03_0086PE,DP03_0086PM,DP03_0087E,DP03_0087M,DP0

In [3]:
#read in 2015 headers
headers = pd.read_csv('s3://bleeding-hearts/rawdata/acs_headers_2015.csv')
headers

Unnamed: 0                                 for  \
0          label        Census API FIPS 'for' clause   
1        concept  Census API Geography Specification   
2  predicateType                            fips-for   
3          group                                 NaN   
4          limit                                   0   
5  predicateOnly                                True   
6     attributes                                 NaN   
7       required                                 NaN   
8         values                                 NaN   

                                   in  \
0         Census API FIPS 'in' clause   
1  Census API Geography Specification   
2                             fips-in   
3                                 NaN   
4                                   0   
5                                True   
6                                 NaN   
7                                 NaN   
8                                 NaN   

                                        ucgid  \
0  Uniform Census Geography Identifier clause   
1          Census API Geography Specification   
2                                       ucgid   
3                                         NaN   
4                                           0   
5                                        True   
6                                         NaN   
7                                         NaN   
8                                         NaN   

                                          DP02_0126E  \
0       Estimate!!ANCESTRY!!Total population!!Danish   
1  SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...   
2                                                int   
3                                               DP02   
4                                                  0   
5                                                NaN   
6                 DP02_0126EA,DP02_0126M,DP02_0126MA   
7                                                NaN   
8                                                NaN   

                                         DP05_0050PE  \
0  Percent!!RACE!!One race!!Native Hawaiian and O...   
1              ACS DEMOGRAPHIC AND HOUSING ESTIMATES   
2                                              float   
3                                               DP05   
4                                                  0   
5                                                NaN   
6              DP05_0050PEA,DP05_0050PM,DP05_0050PMA   
7                                                NaN   
8                                                NaN   

                                          DP04_0047E  \
0  Estimate!!HOUSING TENURE!!Occupied housing uni...   
1                   SELECTED HOUSING CHARACTERISTICS   
2                                                int   
3                                               DP04   
4                                                  0   
5                                                NaN   
6                 DP04_0047EA,DP04_0047M,DP04_0047MA   
7                                                NaN   
8                                                NaN   

                                       DP02PR_0083PE  \
0  Percent!!RESIDENCE 1 YEAR AGO!!Population 1 ye...   
1     SELECTED SOCIAL CHARACTERISTICS IN PUERTO RICO   
2                                              float   
3                                             DP02PR   
4                                                  0   
5                                                NaN   
6        DP02PR_0083PEA,DP02PR_0083PM,DP02PR_0083PMA   
7                                                NaN   
8                                                NaN   

                                       DP02PR_0106PE  \
0  Percent!!WORLD REGION OF BIRTH OF FOREIGN BORN...   
1     SELECTED SOCIAL CHARACTERISTICS IN PUERTO RICO   
2                                              float   
3                                             DP02PR   
4                  

In [4]:
#transpose headers and drop columns that we don't need
headers=headers.T
headers1=headers.drop(columns=[1,2,3,4,5,6,7,8])
headers1

,0
Unnamed: 0,label
for,Census API FIPS 'for' clause
in,Census API FIPS 'in' clause
ucgid,Uniform Census Geography Identifier clause
DP02_0126E,Estimate!!ANCESTRY!!Total population!!Danish
...,...
DP02PR_0001PE,Percent!!HOUSEHOLDS BY TYPE!!Total households
DP03_0039PE,Percent!!INDUSTRY!!Civilian employed populatio...
DP02_0098E,Estimate!!YEAR OF ENTRY!!Native!!Entered 2010 ...
DP04_0095PE,Percent!!SELECTED MONTHLY OWNER COSTS (SMOC)!!...


In [5]:
#turn the headers dataframe into a dictionary
headers_d=headers1.to_dict()
headers_d=headers_d[0]
#loop through the dictionary and replace !! with _
for k,v in headers_d.items():
    headers_d[k]=v.replace('!!','_')
#create a list of all the ACS columns that we will use to cycle through the headers dictionary
cols=list(acs)
len(cols)

1103

Removing attributes from the headers. See https://api.census.gov/data/2018/acs/acs1/profile/variables.html that the actual variables end with "E" or "PE". The attributes end with a variety of letter combinations. At the lowest level, the attributes can be removed by searching for "EA" and "M" to remove them.

In [6]:
cols=[i for i in cols if "EA" not in i]
cols=[i for i in cols if "M" not in i]
len(cols)

280

In [7]:
#Creating another dictionary. Looping through the list of column names from the ACS dataset and if they are in the headers dictionary, placing them in the new dictionary that will contain only headers in the ACS dataset.
dp03={}
for k, v in headers_d.items():
    if k in cols:
        dp03[k]=v
len(dp03)

276

In [8]:
#Removing the attributes.
acs=acs[cols]
#renaming the columns
acs.rename(columns=dp03,inplace=True)
acs.head()

/Users/sarahmartin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,label,Geography,Estimate_EMPLOYMENT STATUS_Population 16 years and over,Percent_EMPLOYMENT STATUS_Population 16 years and over,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force,Percent_EMPLOYMENT STATUS_Population 16 years and over_In labor force,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force,Percent_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Employed,Percent_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Employed,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Unemployed,Percent_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Unemployed,Estimate_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Armed Forces,Percent_EMPLOYMENT STATUS_Population 16 years and over_In labor force_Armed Forces,Estimate_EMPLOYMENT STATUS_Population 16 years and over_Not in labor force,Percent_EMPLOYMENT STATUS_Population 16 years and over_Not in labor force,Estimate_EMPLOYMENT STATUS_Civilian labor force,Percent_EMPLOYMENT STATUS_Civilian labor force,Estimate_EMPLOYMENT STATUS_Civilian labor force_Unemployment Rate,Percent_EMPLOYMENT STATUS_Civilian labor force_Unemployment Rate,Estimate_EMPLOYMENT STATUS_Females 16 years and over,Percent_EMPLOYMENT STATUS_Females 16 years and over,Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force,Percent_EMPLOYMENT STATUS_Females 16 years and over_In labor force,Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force,Percent_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force,Estimate_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force_Employed,Percent_EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force_Employed,Estimate_EMPLOYMENT STATUS_Own children of the householder under 6 years,Percent_EMPLOYMENT STATUS_Own children of the householder under 6 years,Estimate_EMPLOYMENT STATUS_Own children of the householder under 6 years_All parents in family in labor force,Percent_EMPLOYMENT STATUS_Own children of the householder under 6 years_All parents in family in labor force,Estimate_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years,Percent_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years,Estimate_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years_All parents in family in labor force,Percent_EMPLOYMENT STATUS_Own children of the householder 6 to 17 years_All parents in family in labor force,Estimate_COMMUTING TO WORK_Workers 16 years and over,Percent_COMMUTING TO WORK_Workers 16 years and over,"Estimate_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- drove alone","Percent_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- drove alone","Estimate_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- carpooled","Percent_COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- carpooled",Estimate_COMMUTING TO WORK_Workers 16 years and over_Public transportation (excluding taxicab),Percent_COMMUTING TO WORK_Workers 16 years and over_Public transportation (excluding taxicab),Estimate_COMMUTING TO WORK_Workers 16 years and over_Walked,Percent_COMMUTING TO WORK_Workers 16 years and over_Walked,Estimate_COMMUTING TO WORK_Workers 16 years and over_Other means,Percent_COMMUTING TO WORK_Workers 16 years and over_Other means,Estimate_COMMUTING TO WORK_Workers 16 years and over_Worked at home,Percent_COMMUTING TO WORK_Workers 16 years and over_Worked at home,Estimate_COMMUTING TO WORK_Mean travel time to work (minutes),Percent_COMMUTING TO WORK_Mean travel time to work (minutes),Estimate_OCCUPATION_Civilian employed population 16 years and over,Percent_OCCUPATION_Civilian employed popula

## Create a FIPS Code variable in the ACS dataset
The COI data has a FIPS code. In the ACS data, this information is captured in three columns: State, County, and Tract. Merging State, County, and Tract columns into a single variable that can be merged with the COI GEOID in later steps. See support for merge here: https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html

In [9]:
#cast as strings and then join
for i in ['state','county','tract']:
    acs[i] = acs[i].astype(str)
acs['FIPS'] = acs[['state', 'county', 'tract']].agg(''.join, axis=1)
acs['FIPS']=acs['FIPS'].astype(int)
acs[['state','county','tract','FIPS']]

<ipython-input-9-b9ccf8423fb0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acs[i] = acs[i].astype(str)


,state,county,tract,FIPS
0,1,35,960200,135960200
1,1,35,960300,135960300
2,1,35,960400,135960400
3,1,35,960500,135960500
4,1,35,960600,135960600
...,...,...,...,...
207033,55,79,13500,557913500
207034,55,79,16100,557916100
207035,55,79,101100,5579101100
207036,55,79,170700,5579170700


In [10]:
#dropping columns we don't need anymore
dropcols = ['label',
           'Geography',
           'state',
           'county',
           'tract']
acs.drop(columns=dropcols,inplace=True)

## Split ACS data int estimate and percent datasets
The ACS dataset now has duplicate columns, one with the data formatted as an estimate and another with it formatted as a percet. We also remove the words "estimate" and "percent" from each column name.

In [11]:
#creating a list of columns and splitting the list into two: one for percent and one for estimate
cols = list(acs)
est = [i for i in cols if "Percent" not in i]
pct = [i for i in cols if "Estimate" not in i]
#using the est and pct lists to split the acs dataframe into two df's: one for percent and one for estimate
acs_est = acs[est]
acs_pct = acs[pct]

In [12]:
acs_est.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207038 entries, 0 to 207037
Columns: 139 entries, Estimate_EMPLOYMENT STATUS_Population 16 years and over to FIPS
dtypes: float64(1), int64(138)
memory usage: 219.6 MB


In [13]:
acs_pct.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207038 entries, 0 to 207037
Columns: 139 entries, Percent_EMPLOYMENT STATUS_Population 16 years and over to FIPS
dtypes: float64(102), int64(37)
memory usage: 219.6 MB


In [14]:
#define a function to save data to S3
with open('/Volumes/Key/aws_keys.json') as f:
    creds = json.load(f)
def s3upload(name,df,folder):
    x = df.to_csv(None).encode()
    fs = s3fs.S3FileSystem(key=creds['AWSAccessKeyId'], secret=creds['AWSSecretKey'])
    with fs.open('s3://bleeding-hearts/'+folder + name + '.csv', 'wb') as f:
        f.write(x)
    print('saved')

In [15]:
#removing "Estimate" and "Percent" from column names
est_clean = [i.replace('Estimate_','') for i in est]
pct_clean = [i.replace('Percent_','') for i in pct]
#creating a dictionary for each est and pct that we will use to rename the columns without "Estimate" and "Percent"
est_d=dict(zip(est,est_clean))
pct_d=dict(zip(pct,pct_clean))
#renaming columns
acs_est.rename(columns=est_d,inplace=True)
acs_pct.rename(columns=pct_d,inplace=True)
#upload to S3
s3upload('acs_est',acs_est,'workingdata/')

/Users/sarahmartin/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4300: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


saved


In [16]:
s3upload('acs_pct',acs_pct,'workingdata/')

saved


## Drop COI columns that we do not need 
Open the COI Index file. This file contains the COI indices that we will use as targets. The file includes the indicies normed at various geographic levels. We decided to assess the data at the nationally-normed level, so in the following steps we drop the columns that contain data normed at other geographic level. We also drop z-scores.

In [17]:
coi_index = pd.read_csv('s3://bleeding-hearts/rawdata/coi_index.csv')
coi_index.head()

,Unnamed: 0,geoid,year,in100,msaid15,msaname15,countyfips,statefips,stateusps,pop,z_ED_nat,z_HE_nat,z_SE_nat,z_COI_nat,c5_ED_nat,c5_HE_nat,c5_SE_nat,c5_COI_nat,r_ED_nat,r_HE_nat,r_SE_nat,r_COI_nat,c5_ED_stt,c5_HE_stt,c5_SE_stt,c5_COI_stt,r_ED_stt,r_HE_stt,r_SE_stt,r_COI_stt,c5_ED_met,c5_HE_met,c5_SE_met,c5_COI_met,r_ED_met,r_HE_met,r_SE_met,r_COI_met
0,0,1001020100,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,424,0.001198,-0.016530,0.046101,0.003759,Moderate,Very Low,Moderate,Moderate,55,20,54,49,High,Low,High,High,67,33,67,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1001020100,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,414,0.008932,0.035614,0.146657,0.021866,Moderate,High,High,High,59,66,75,71,High,Very High,Very High,Very High,71,92,83,84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1001020200,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,497,0.013306,-0.022400,-0.133538,-0.016473,High,Very Low,Low,Low,62,17,23,27,High,Low,Low,Low,73,28,31,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1001020200,2015,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,508,-0.003518,-0.012027,-0.169899,-0.021479,Moderate,Low,Very Low,Low,53,23,18,23,High,Low,Low,Low,64,39,26,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1001020300,2010,0.0,33860.0,"Montgomery, AL Metro Area",1001,1,AL,878,-0.014533,-0.026710,0.102040,0.007462,Moderate,Very Low,High,Moderate,46,15,66,54,Moderate,Low,High,High,59,23,77,69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#dropping columns that we don't need
cols=list(coi_index)
keep=[i for i in cols if 'nat' in i]
keep=[i for i in keep if 'z' not in i]
keep.insert(0,'geoid')
keep.insert(1,'year')
coi_index=coi_index[keep]
coi_index.head()

,geoid,year,c5_ED_nat,c5_HE_nat,c5_SE_nat,c5_COI_nat,r_ED_nat,r_HE_nat,r_SE_nat,r_COI_nat
0,1001020100,2010,Moderate,Very Low,Moderate,Moderate,55,20,54,49
1,1001020100,2015,Moderate,High,High,High,59,66,75,71
2,1001020200,2010,High,Very Low,Low,Low,62,17,23,27
3,1001020200,2015,Moderate,Low,Very Low,Low,53,23,18,23
4,1001020300,2010,Moderate,Very Low,High,Moderate,46,15,66,54


## COI Headers

Similar to the ACS data, the COI headers are codes. The data dictionary defines the codes in the dataset. Read in a PDF version of this webpage, which includes the data dictionary: https://data.diversitydatakids.org/dataset/coi20-child-opportunity-index-2-0-database/resource/080cfe52-90aa-4925-beaa-90efb04ab7fb. Use the data dictionary to replace the headers with descriptions.

In [19]:
tbl_lst = tabula.read_pdf('/Users/sarahmartin/Documents/GT/COI_index_headers.pdf',
                         pages="all",
                         multiple_tables=True,
                         stream=True)
print(len(tbl_lst))

6


In [20]:
#concat tables from the above list and drop blank/unnessary rows
index_head = pd.concat([tbl_lst[2],tbl_lst[3],tbl_lst[4],tbl_lst[5]])
index_head.columns = index_head.iloc[0]
index_head.drop(index_head.index[0],inplace=True)
index_head = index_head[index_head['Label'].notna()]
index_head.reset_index(inplace=True,drop=True)
index_head.head(25)

,Column,Type,Label,Description
0,geoid,text,2010 census tract FIPS codes,NaN
1,year,numeric,Year of observation (2010 or,NaN
2,NaN,NaN,2015),NaN
3,in100,numeric,In one of the 100 largest metro,This variable is equal to 1 if the
4,NaN,NaN,areas,census tract is located in one of
5,msaid15,text,Metro/Micro Area (2015) FIPS,NaN
6,NaN,NaN,Code,NaN
7,msaname15,text,Metro/Micro Area (2015) Name,NaN
8,statefips,text,State FIPS Code,NaN
9,stateusps,text,State USPS Code,NaN


In [21]:
#transpose the dataset and drop unnessary columns
transp = index_head.drop(columns=['Type','Description'])
lst = list(transp.index[transp['Column'].isnull()])
transp.drop(columns=['Column'],inplace=True)
#transpose the dataframe back. Merge sequential rows that read in separately where line breaks occurred in the PDF.
transp = transp.T
for i in reversed(lst):
    transp[i-1] = transp[i-1] + ' ' + transp[i]   
#drop unnessary columns and transpose again.
transp.drop(columns=lst,inplace=True)
transp = transp.T
transp.head()

,Label
0,2010 census tract FIPS codes
1,Year of observation (2010 or 2015)
3,In one of the 100 largest metro areas
5,Metro/Micro Area (2015) FIPS Code
7,Metro/Micro Area (2015) Name


In [22]:
#drop columns and merge index_head and transp
index_head = index_head[index_head['Column'].notna()]
index_head.drop(columns='Label',inplace=True)
index_head = index_head.merge(transp, how='outer', left_index=True, right_index=True)
index_head

,Column,Type,Description,Label
0,geoid,text,NaN,2010 census tract FIPS codes
1,year,numeric,NaN,Year of observation (2010 or 2015)
3,in100,numeric,This variable is equal to 1 if the,In one of the 100 largest metro areas
5,msaid15,text,NaN,Metro/Micro Area (2015) FIPS Code
7,msaname15,text,NaN,Metro/Micro Area (2015) Name
8,statefips,text,NaN,State FIPS Code
9,stateusps,text,NaN,State USPS Code
10,pop,numeric,NaN,"Number of children aged 0-17, ACS (2008-12 for..."
13,z_ED_nat,numeric,Weighted average of education,"Z-scores, education domain, nationally-normed"
15,z_HE_nat,numeric,Weighted average of health and,"Z-scores, health and environment domain, natio..."


In [23]:
#drop columns, reset index, and transpose -- preparing data to turn it into a dictionary.
index_head.drop(columns=['Type','Description'],inplace=True)
index_head.set_index('Column',inplace=True)
index_head=index_head.T
index_head

Column,geoid,year,in100,msaid15,msaname15,statefips,stateusps,pop,z_ED_nat,z_HE_nat,z_SE_nat,z_COI_nat,c5_ED_nat,c5_HE_nat,c5_SE_nat,c5_COI_nat,r_ED_nat,r_HE_nat,r_SE_nat,r_COI_nat,c5_ED_stt,c5_HE_stt,c5_SE_stt,c5_COI_stt,r_ED_stt,r_HE_stt,r_SE_stt,r_COI_stt,c5_ED_met,c5_HE_met,c5_SE_met,c5_COI_met,r_ED_met,r_HE_met,r_SE_met,r_COI_met
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Label,2010 census tract FIPS codes,Year of observation (2010 or 2015),In one of the 100 largest metro areas,Metro/Micro Area (2015) FIPS Code,Metro/Micro Area (2015) Name,State FIPS Code,State USPS Code,"Number of children aged 0-17, ACS (2008-12 for...","Z-scores, education domain, nationally-normed","Z-scores, health and environment domain, natio...","Z-scores, social and economic domain, national...","Z-scores, overall COI, nationally-normed","Child Opportunity Levels, education domain, na...","Child Opportunity Levels, health and environme...","Child Opportunity Levels, social and economic ...","Child Opportunity Levels, overall COI, nationa...","Child Opportunity Scores, education domain, na...","Child Opportunity Scores, health and environme...","Child Opportunity Scores, social and economic ...","Child Opportunity Scores, overall COI, nationa...","Child Opportunity Levels, education domain, st...","Child Opportunity Levels, health and environme...","Child Opportunity Levels, social and economic ...","Child Opportunity Levels, overall COI, state-n...","Child Opportunity Scores, education domain, st...","Child Opportunity Scores, health and environme...","Child Opportunity Scores, social and economic ...","Child Opportunity Scores, overall COI, state-n...","Child Opportunity Levels, education domain, me...","Child Opportunity Levels, health and environme...","Child Opportunity Levels, social and economic ...","Child Opportunity Levels, overall COI, metro-n...","Child Opportunity Scores, education domain, me...","Child Opportunity Scores, health and environme...","Child Opportunity Scores, social and economic ...","Child Opportunity Scores, overall COI, metro-n..."


In [24]:
#transform data into a dictionary
index_head_d = index_head.to_dict('records')
index_head_d = index_head_d[0]
index_head_d

{'geoid': '2010 census tract FIPS codes',
 'year': 'Year of observation (2010 or 2015)',
 'in100': 'In one of the 100 largest metro areas',
 'msaid15': 'Metro/Micro Area (2015) FIPS Code',
 'msaname15': 'Metro/Micro Area (2015) Name',
 'statefips': 'State FIPS Code',
 'stateusps': 'State USPS Code',
 'pop': 'Number of children aged 0-17, ACS (2008-12 for 2010, 2013-17 for 2015)',
 'z_ED_nat': 'Z-scores, education domain, nationally-normed',
 'z_HE_nat': 'Z-scores, health and environment domain, nationally-normed',
 'z_SE_nat': 'Z-scores, social and economic domain, nationally-normed',
 'z_COI_nat': 'Z-scores, overall COI, nationally-normed',
 'c5_ED_nat': 'Child Opportunity Levels, education domain, nationally- normed',
 'c5_HE_nat': 'Child Opportunity Levels, health and environment domain, nationally-normed',
 'c5_SE_nat': 'Child Opportunity Levels, social and economic domain, nationally-normed',
 'c5_COI_nat': 'Child Opportunity Levels, overall COI, nationally-normed',
 'r_ED_nat': '

In [25]:
#use the dictionary to rename the columns
coi_index.rename(columns=index_head_d,inplace=True)
coi_index

,2010 census tract FIPS codes,Year of observation (2010 or 2015),"Child Opportunity Levels, education domain, nationally- normed","Child Opportunity Levels, health and environment domain, nationally-normed","Child Opportunity Levels, social and economic domain, nationally-normed","Child Opportunity Levels, overall COI, nationally-normed","Child Opportunity Scores, education domain, nationally- normed","Child Opportunity Scores, health and environment domain, nationally-normed","Child Opportunity Scores, social and economic domain, nationally-normed","Child Opportunity Scores, overall COI, nationally-normed"
0,1001020100,2010,Moderate,Very Low,Moderate,Moderate,55,20,54,49
1,1001020100,2015,Moderate,High,High,High,59,66,75,71
2,1001020200,2010,High,Very Low,Low,Low,62,17,23,27
3,1001020200,2015,Moderate,Low,Very Low,Low,53,23,18,23
4,1001020300,2010,Moderate,Very Low,High,Moderate,46,15,66,54
...,...,...,...,...,...,...,...,...,...,...
144403,56043000302,2015,High,Moderate,Moderate,Moderate,67,56,58,60
144404,56045951100,2010,Low,High,Low,Low,38,80,31,37
144405,56045951100,2015,Moderate,High,Low,Moderate,49,68,35,41
144406,56045951300,2010,Moderate,High,Moderate,Moderate,52,70,48,52


In [26]:
s3upload('coi_index',coi_index,'workingdata/')

saved


## Merge Datasets
Now we merge the ACS estimate data with the COI index data and the ACS percent data with the COI index data. We merge the datasets on both the FIPS code and the year.

In [27]:
#merge on both the FIPS code and the year
merge_pct = pd.merge(acs_pct, 
                  coi_index,
                 left_on=['FIPS','year'],
                  right_on=['2010 census tract FIPS codes','Year of observation (2010 or 2015)'])
merge_est = pd.merge(acs_est, 
                  coi_index,
                 left_on=['FIPS','year'],
                  right_on=['2010 census tract FIPS codes','Year of observation (2010 or 2015)'])
#drop duplicate cols
merge_pct.drop(columns=['2010 census tract FIPS codes','Year of observation (2010 or 2015)'],inplace=True)
merge_est.drop(columns=['2010 census tract FIPS codes','Year of observation (2010 or 2015)'],inplace=True)

In [28]:
merge_pct.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18897 entries, 0 to 18896
Columns: 147 entries, EMPLOYMENT STATUS_Population 16 years and over to Child Opportunity Scores, overall COI, nationally-normed
dtypes: float64(102), int64(41), object(4)
memory usage: 21.3+ MB


In [29]:
merge_est.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18897 entries, 0 to 18896
Columns: 147 entries, EMPLOYMENT STATUS_Population 16 years and over to Child Opportunity Scores, overall COI, nationally-normed
dtypes: float64(1), int64(142), object(4)
memory usage: 21.3+ MB


In [30]:
merge_pct.head()

,EMPLOYMENT STATUS_Population 16 years and over,EMPLOYMENT STATUS_Population 16 years and over_In labor force,EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force,EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Employed,EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Unemployed,EMPLOYMENT STATUS_Population 16 years and over_In labor force_Armed Forces,EMPLOYMENT STATUS_Population 16 years and over_Not in labor force,EMPLOYMENT STATUS_Civilian labor force,EMPLOYMENT STATUS_Civilian labor force_Unemployment Rate,EMPLOYMENT STATUS_Females 16 years and over,EMPLOYMENT STATUS_Females 16 years and over_In labor force,EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force,EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force_Employed,EMPLOYMENT STATUS_Own children of the householder under 6 years,EMPLOYMENT STATUS_Own children of the householder under 6 years_All parents in family in labor force,EMPLOYMENT STATUS_Own children of the householder 6 to 17 years,EMPLOYMENT STATUS_Own children of the householder 6 to 17 years_All parents in family in labor force,COMMUTING TO WORK_Workers 16 years and over,"COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- drove alone","COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- carpooled",COMMUTING TO WORK_Workers 16 years and over_Public transportation (excluding taxicab),COMMUTING TO WORK_Workers 16 years and over_Walked,COMMUTING TO WORK_Workers 16 years and over_Other means,COMMUTING TO WORK_Workers 16 years and over_Worked at home,COMMUTING TO WORK_Mean travel time to work (minutes),OCCUPATION_Civilian employed population 16 years and over,"OCCUPATION_Civilian employed population 16 years and over_Management, business, science, and arts occupations",OCCUPATION_Civilian employed population 16 years and over_Service occupations,OCCUPATION_Civilian employed population 16 years and over_Sales and office occupations,"OCCUPATION_Civilian employed population 16 years and over_Natural resources, construction, and maintenance occupations","OCCUPATION_Civilian employed population 16 years and over_Production, transportation, and material moving occupations",INDUSTRY_Civilian employed population 16 years and over,"INDUSTRY_Civilian employed population 16 years and over_Agriculture, forestry, fishing and hunting, and mining",INDUSTRY_Civilian employed population 16 years and over_Construction,INDUSTRY_Civilian employed population 16 years and over_Manufacturing,INDUSTRY_Civilian employed population 16 years and over_Wholesale trade,INDUSTRY_Civilian employed population 16 years and over_Retail trade,"INDUSTRY_Civilian employed population 16 years and over_Transportation and warehousing, and utilities",INDUSTRY_Civilian employed population 16 years and over_Information,"INDUSTRY_Civilian employed population 16 years and over_Finance and insurance, and real estate and rental and leasing","INDUSTRY_Civilian employed population 16 years and over_Professional, scientific, and management, and administrative and waste management services","INDUSTRY_Civilian employed population 16 years and over_Educational services, and health care and social assistance","INDUSTRY_Civilian employed population 16 years and over_Arts, entertainment, and recreation, and accommodation and food services","INDUSTRY_Civilian employed population 16 years and over_Other services, except public administration",INDUSTRY_Civilian employed population 16 years and over_Public administration,CLASS OF WORKER_Civilian employed population 16 years and over,CLASS OF WORKER_Civilian employed population 16 years and over_Private wage and salary workers,CLASS OF WORKER_Civilian employed population 16 years and over_Government workers,CLASS OF WORKER_Civilian employed population 16 years and over_Self-employed in own not incorporated business workers,CLASS OF WORKER_Civil

In [31]:
merge_est.head()

,EMPLOYMENT STATUS_Population 16 years and over,EMPLOYMENT STATUS_Population 16 years and over_In labor force,EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force,EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Employed,EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Unemployed,EMPLOYMENT STATUS_Population 16 years and over_In labor force_Armed Forces,EMPLOYMENT STATUS_Population 16 years and over_Not in labor force,EMPLOYMENT STATUS_Civilian labor force,EMPLOYMENT STATUS_Civilian labor force_Unemployment Rate,EMPLOYMENT STATUS_Females 16 years and over,EMPLOYMENT STATUS_Females 16 years and over_In labor force,EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force,EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force_Employed,EMPLOYMENT STATUS_Own children of the householder under 6 years,EMPLOYMENT STATUS_Own children of the householder under 6 years_All parents in family in labor force,EMPLOYMENT STATUS_Own children of the householder 6 to 17 years,EMPLOYMENT STATUS_Own children of the householder 6 to 17 years_All parents in family in labor force,COMMUTING TO WORK_Workers 16 years and over,"COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- drove alone","COMMUTING TO WORK_Workers 16 years and over_Car, truck, or van -- carpooled",COMMUTING TO WORK_Workers 16 years and over_Public transportation (excluding taxicab),COMMUTING TO WORK_Workers 16 years and over_Walked,COMMUTING TO WORK_Workers 16 years and over_Other means,COMMUTING TO WORK_Workers 16 years and over_Worked at home,COMMUTING TO WORK_Mean travel time to work (minutes),OCCUPATION_Civilian employed population 16 years and over,"OCCUPATION_Civilian employed population 16 years and over_Management, business, science, and arts occupations",OCCUPATION_Civilian employed population 16 years and over_Service occupations,OCCUPATION_Civilian employed population 16 years and over_Sales and office occupations,"OCCUPATION_Civilian employed population 16 years and over_Natural resources, construction, and maintenance occupations","OCCUPATION_Civilian employed population 16 years and over_Production, transportation, and material moving occupations",INDUSTRY_Civilian employed population 16 years and over,"INDUSTRY_Civilian employed population 16 years and over_Agriculture, forestry, fishing and hunting, and mining",INDUSTRY_Civilian employed population 16 years and over_Construction,INDUSTRY_Civilian employed population 16 years and over_Manufacturing,INDUSTRY_Civilian employed population 16 years and over_Wholesale trade,INDUSTRY_Civilian employed population 16 years and over_Retail trade,"INDUSTRY_Civilian employed population 16 years and over_Transportation and warehousing, and utilities",INDUSTRY_Civilian employed population 16 years and over_Information,"INDUSTRY_Civilian employed population 16 years and over_Finance and insurance, and real estate and rental and leasing","INDUSTRY_Civilian employed population 16 years and over_Professional, scientific, and management, and administrative and waste management services","INDUSTRY_Civilian employed population 16 years and over_Educational services, and health care and social assistance","INDUSTRY_Civilian employed population 16 years and over_Arts, entertainment, and recreation, and accommodation and food services","INDUSTRY_Civilian employed population 16 years and over_Other services, except public administration",INDUSTRY_Civilian employed population 16 years and over_Public administration,CLASS OF WORKER_Civilian employed population 16 years and over,CLASS OF WORKER_Civilian employed population 16 years and over_Private wage and salary workers,CLASS OF WORKER_Civilian employed population 16 years and over_Government workers,CLASS OF WORKER_Civilian employed population 16 years and over_Self-employed in own not incorporated business workers,CLASS OF WORKER_Civil

In [32]:
for i in [merge_est,merge_pct]:
    #changing FIPS to a string so we can onehot encode it later
    i['FIPS']=i['FIPS'].astype('string')
    #there are some duplicate rows; dropping them
    i.drop_duplicates(inplace=True)
    print(i.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18208 entries, 0 to 18896
Columns: 147 entries, EMPLOYMENT STATUS_Population 16 years and over to Child Opportunity Scores, overall COI, nationally-normed
dtypes: float64(1), int64(141), object(4), string(1)
memory usage: 20.6+ MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 18208 entries, 0 to 18896
Columns: 147 entries, EMPLOYMENT STATUS_Population 16 years and over to Child Opportunity Scores, overall COI, nationally-normed
dtypes: float64(102), int64(40), object(4), string(1)
memory usage: 20.6+ MB
None


In [33]:
s3upload('merge_pct',merge_pct,'workingdata/')

saved


In [34]:
s3upload('merge_est',merge_est,'workingdata/')

saved


## Assess and mitigate missing data
Some columns have values that we interpret to be missing; they are formatted like:
* -888888
* -666666
* -222222
<br>
We identified the missing values by searching for any values less than 0 and replacing it with nan. There are some columns where all the values are missing and other columns that contain real data in addition to the missing values. First we drop the columns where all of the values are missing. 

In [35]:
cols=list(merge_est)
#remove string cols from list
cols=cols[:-9]
all_miss = []
for c in cols:
    if merge_est[c].dtypes != object:
        merge_est.loc[merge_est[c] < 0, c] = np.nan
    if len(merge_est) - merge_est[c].count() == len(merge_est):
        all_miss.append(c)
merge_est.drop(columns=all_miss,inplace=True)
print(len(all_miss),'columns were removed from the est dataset because all values were missing')

20 columns were removed from the est dataset because all values were missing


In [36]:
cols=list(merge_pct)
#remove string cols from list
cols=cols[:-9]
all_miss = []
for c in cols:
    if merge_pct[c].dtypes != object:
        merge_pct.loc[merge_pct[c] < 0, c] = np.nan
    if len(merge_pct) - merge_pct[c].count() == len(merge_pct):
        all_miss.append(c)
merge_pct.drop(columns=all_miss,inplace=True)
print(len(all_miss),'columns were removed from the pct dataset because all values were missing')

16 columns were removed from the pct dataset because all values were missing


In [37]:
#define a function to see the percentage of missing values in the remaining columns. 
def assess_missing(df):
    missing_dict = {}
    cols=list(df)
    for c in cols:
        if len(df) - df[c].count() > 0:
             missing_dict[c] = ((len(df) - df[c].count())/len(df))*100
    return missing_dict

In [38]:
#see that the columns with the most missing values have about 50% remaining
est_missing = assess_missing(merge_est)
est_missing

{'COMMUTING TO WORK_Mean travel time to work (minutes)': 0.35149384885764495,
 'INCOME AND BENEFITS (IN 2015 INFLATION-ADJUSTED DOLLARS)_Total households_Median household income (dollars)': 0.19771528998242532,
 'INCOME AND BENEFITS (IN 2015 INFLATION-ADJUSTED DOLLARS)_Total households_Mean household income (dollars)': 0.21968365553602814,
 'INCOME AND BENEFITS (IN 2015 INFLATION-ADJUSTED DOLLARS)_With earnings_Mean earnings (dollars)': 0.2251757469244288,
 'INCOME AND BENEFITS (IN 2015 INFLATION-ADJUSTED DOLLARS)_With Social Security_Mean Social Security income (dollars)': 0.5437170474516696,
 'INCOME AND BENEFITS (IN 2015 INFLATION-ADJUSTED DOLLARS)_With retirement income_Mean retirement income (dollars)': 0.8018453427065025,
 'INCOME AND BENEFITS (IN 2015 INFLATION-ADJUSTED DOLLARS)_With Supplemental Security Income_Mean Supplemental Security Income (dollars)': 10.209797891036906,
 'INCOME AND BENEFITS (IN 2015 INFLATION-ADJUSTED DOLLARS)_With cash public assistance income_Mean cash

In [39]:
#see that the columns with the most missing values have about 50% remaining
pct_missing = assess_missing(merge_pct)
pct_missing

{'EMPLOYMENT STATUS_Population 16 years and over_In labor force': 0.027460456942003518,
 'EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force': 0.027460456942003518,
 'EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Employed': 0.027460456942003518,
 'EMPLOYMENT STATUS_Population 16 years and over_In labor force_Civilian labor force_Unemployed': 0.027460456942003518,
 'EMPLOYMENT STATUS_Population 16 years and over_In labor force_Armed Forces': 0.027460456942003518,
 'EMPLOYMENT STATUS_Population 16 years and over_Not in labor force': 0.027460456942003518,
 'EMPLOYMENT STATUS_Civilian labor force_Unemployment Rate': 0.04942882249560633,
 'EMPLOYMENT STATUS_Females 16 years and over_In labor force': 0.04393673110720562,
 'EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force': 0.04393673110720562,
 'EMPLOYMENT STATUS_Females 16 years and over_In labor force_Civilian labor force_Employed': 0.043936

In [40]:
#upload est dataset to S3
s3upload('est_clean',merge_est,'workingdata/')

saved


In [41]:
#upload pct dataset to S3
s3upload('pct_clean',merge_pct,'workingdata/')

saved


## Overall COI only
Save copies of the est and pct datasets with only the overall COI level for the target variable.

In [45]:
drop = ['Child Opportunity Levels, education domain, nationally- normed',
 'Child Opportunity Levels, health and environment domain, nationally-normed',
 'Child Opportunity Levels, social and economic domain, nationally-normed',
 'Child Opportunity Scores, education domain, nationally- normed',
 'Child Opportunity Scores, health and environment domain, nationally-normed',
 'Child Opportunity Scores, social and economic domain, nationally-normed',
 'Child Opportunity Scores, overall COI, nationally-normed']
merge_est.drop(columns=drop,inplace=True)
merge_pct.drop(columns=drop,inplace=True)
#upload to S3
s3upload('est_overall',merge_est,'workingdata/')

saved


In [46]:
s3upload('pct_overall',merge_pct,'workingdata/')

saved
